<a href="https://colab.research.google.com/github/vitthal-bhandari/incremental-semi-supervised-training/blob/master/Baselines/Multilingual_Baselines_Task_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializing environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct  6 13:44:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# !pip install datasets
# !pip install transformers
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install wandb
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
API_KEY = 'ddb1a3a21db80aada85fe7d1e39a95d2abbdc426'

In [5]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import r_regression
import pandas as pd
import numpy as np
import torch

import wandb
wandb.login()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vitthal. Use `wandb login --relogin` to force relogin


True

# Reading & splitting data

In [36]:
#Reading untouched data (non-preprocessed, non-augmented)

dataset = pd.read_csv("train_all_tasks.csv")
dataset.rename(columns={'label_sexist': 'labels'}, inplace=True)

print(len(dataset))

14000


In [37]:
#Reading untouched data (non-preprocessed, non-augmented)

dev_df = pd.read_csv("dev_task_a_entries.csv")

print(len(dev_df))

2000


In [38]:
dataset.head()

,rewire_id,text,labels,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none


In [39]:
dev_df.head()

,rewire_id,text
0,sexism2022_english-11999,I do not try to hard. I do what guys who get w...
1,sexism2022_english-9895,That sucks. We elected a Navy Seal in Missouri...
2,sexism2022_english-11623,Walk away and never look back. If she reaches ...
3,sexism2022_english-19035,basically I placed more value on her than I sh...
4,sexism2022_english-895,"NYT and Asian ""part ways."" But why on earth wa..."


In [40]:

def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.7, frac_val=0.3,
                                         random_state=45):

    if frac_train + frac_val != 1.0:
        raise ValueError('fractions %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_val, y_train, y_val = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val)

    return df_train, df_val

In [41]:
df_train, df_val = \
    split_stratified_into_train_val_test(dataset, stratify_colname='labels', frac_train=0.80, frac_val=0.20)

In [42]:
df_train["labels"].value_counts()

not sexist    8482
sexist        2718
Name: labels, dtype: int64

In [43]:
df_val["labels"].value_counts()

not sexist    2120
sexist         680
Name: labels, dtype: int64

# Instantiating model

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
    
# result, model_outputs, wrong_predictions = model.eval_model(df_dev, f1=f1_multiclass, acc=accuracy_score)

# result

In [ ]:
import wandb

wandb.init(project="Semeval Task 9 - Multilingual Tweet Intimacy analysis", entity="vitthal")

sweep_config = {
    "method": "bayes",
    'name': 'sweep_06_Oct_train_dev_split',
    "metric": {"name": "f1score", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"values": [1, 2]},
        "learning_rate": {"values": [1e-5, 2e-5, 4e-5, 1e-4, 4e-4]},
    },
}

sweep_id = wandb.sweep(sweep_config, project="SemEval Task 10 - Explainable Detection of Online Sexism", entity="vitthal")

import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "evaluate_during_training": True,
    "manual_seed": 4,
    "use_multiprocessing": True,
    "train_batch_size": 16,
    "eval_batch_size": 8,
    "labels_list": ["not sexist", "sexist"],
    "learning_rate": 4e-5,
    "num_train_epochs": 2,
    "wandb_project": "SemEval Task 10 - Explainable Detection of Online Sexism",
    }

def train():
  # Initialize a new wandb run
  wandb.init()

  # Create a ClassificationModel
  model = ClassificationModel(
      "bert", 
      "bert-base-multilingual-cased",
      use_cuda=True,
      args=train_args,
      sweep_config=wandb.config,
  )
  
  # Train the model
  model.train_model(
      df_train, 
      eval_df=df_val,
      f1score=lambda truth, predictions: f1_multiclass(
        truth, [round(p) for p in predictions]
    ),)
  

  # Sync wandb
  wandb.join()

wandb: Currently logged in as: vitthal. Use `wandb login --relogin` to force relogin


Create sweep with ID: n3g9yv6z
Sweep URL: https://wandb.ai/vitthal/SemEval%20Task%2010%20-%20Explainable%20Detection%20of%20Online%20Sexism/sweeps/n3g9yv6z


# Run sweep

In [ ]:
wandb.agent(sweep_id, train)

# Running final MBERT baseline after sweeping

In [14]:
len(dataset)

14000

In [15]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "manual_seed": 4,
    "use_multiprocessing": True,
    "train_batch_size": 16,
    "labels_list": ["not sexist", "sexist"],
    "learning_rate": 2e-5,
    "num_train_epochs": 2,
    }

# Create a ClassificationModel
final_model = ClassificationModel(
    "bert", 
    "bert-base-multilingual-cased",
    use_cuda=True,
    args=train_args,
)
  
# Train the model
final_model.train_model(
    dataset, 
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/14000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/875 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/875 [00:00<?, ?it/s]

(1750, 0.35341076374053954)

In [16]:
# Make predictions with the model
predictions, raw_outputs = final_model.predict(dev_df["text"].to_list())

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [17]:
raw_outputs

array([[ 1.70410156, -1.15527344],
       [ 2.40429688, -1.73925781],
       [ 1.7578125 , -1.14648438],
       ...,
       [ 1.43847656, -1.02148438],
       [-2.19140625,  1.93457031],
       [ 1.98242188, -1.41601562]])

In [18]:
len(predictions)

2000

In [19]:
dev_df['label_pred'] = predictions
dev_df_final = dev_df[['rewire_id', 'label_pred']]
dev_df_final.head(2)

,rewire_id,label_pred
0,sexism2022_english-11999,not sexist
1,sexism2022_english-9895,not sexist


In [21]:
dev_df_final.to_csv('task_a_pred_1.csv', index=False)



---


### mBERT
macro-F1 = 0.7914

lr = 2e-5

num_epochs = 2


---



# Running an xlm baseline with same hyperparams as mBERT

In [22]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "manual_seed": 4,
    "use_multiprocessing": True,
    "train_batch_size": 16,
    "labels_list": ["not sexist", "sexist"],
    "learning_rate": 2e-5,
    "num_train_epochs": 2,
    }

# Create a ClassificationModel
final_model_xlm = ClassificationModel(
    "xlmroberta", 
    "xlm-roberta-base",
    use_cuda=True,
    args=train_args,
)
  
# Train the model
final_model_xlm.train_model(
    dataset, 
)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:487: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/875 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/875 [00:00<?, ?it/s]

(1750, 0.38837559291294643)

In [44]:
# Make predictions with the model
predictions, raw_outputs = final_model_xlm.predict(dev_df["text"].to_list())

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [45]:
raw_outputs

array([[ 1.43261719, -1.98632812],
       [ 1.9609375 , -2.12304688],
       [ 1.5859375 , -1.89648438],
       ...,
       [ 1.54296875, -1.61425781],
       [-1.76367188,  2.19726562],
       [ 1.46289062, -1.55664062]])

In [46]:
len(predictions)

2000

In [47]:
dev_df['label_pred'] = predictions
dev_df_final = dev_df[['rewire_id', 'label_pred']]
dev_df_final.head(2)

,rewire_id,label_pred
0,sexism2022_english-11999,not sexist
1,sexism2022_english-9895,not sexist


In [48]:
dev_df_final.to_csv('task_a_pred_2.csv', index=False)



---


### xlm-roBERTa
macro-F1 = 0.7967

lr = 2e-5

num_epochs = 2


---

